<center> <h1><font size=7> Case Study C</font> </h1> </center>

# Predicting AirBnB Prices - Example Solution - part 1

This notebook contains the data wrangling example answer to Case Study C. There are a wide range of approaches to this task, you may decide to go down a different route than taken here.

In this notebook, data will be cleaned. We will be reading in the data, imputing missing values. Encoding certain values and joining required data. The output of this notebook will be a csv file, which machine learning will be conducted upon in a different notebook. Note: The datasets to be used in this notebook will be airbnb listings for manchester and bristol.

# 1. Import required packages

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce # used to join multiple data sets

# 2.Reading in the data

In [ ]:
manchester_df= pd.read_csv("..//..//data//airbnb//listings//manchester_listings.csv")
greater_manchester_df= pd.read_csv("..//..//data//airbnb//listings//greater_manchester_listings.csv")
bristol_df= pd.read_csv("..//..//data//airbnb//listings//bristol_listings.csv")

In [ ]:
# Just so I can identity the cities later I will create a new column called city
manchester_df['city'] = 'Manchester'

bristol_df['city'] = 'Bristol'

greater_manchester_df['city'] = 'Manchester'

# 3.Check to see which columns are the same

In [ ]:
column_differences_bm = manchester_df.columns.difference(bristol_df.columns)
column_differences_bm

#### All good, we wont be using most of those columns.

# 4.Imputing Data 1 (There will be a 2 later when I bfill the bedrooms)

In [ ]:
# greater manhester and bristol missing bathroom data but has bathroom text so will take first vlaue
greater_manchester_df['bathrooms'] = greater_manchester_df['bathrooms_text'].str[0:1]
bristol_df['bathrooms'] = bristol_df['bathrooms_text'].str[0:1]

# 5.Selecting the columns we want and concatting bristol and manchester dataset

In [ ]:
columns = ['id', 'neighbourhood_cleansed', 'city', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
           'bathrooms', 'bedrooms', 'price', 'minimum_nights', 'maximum_nights',
           'availability_365', 'number_of_reviews', 'reviews_per_month',
           'host_is_superhost', 'amenities']


manchester_df = manchester_df[columns]
greater_manchester_df = greater_manchester_df[columns]
bristol_df = bristol_df[columns]

In [ ]:
combined_df = pd.concat([manchester_df, bristol_df, greater_manchester_df])

In [ ]:
combined_df

#### As we can we see there is a dollar sign in front of the price, which we need to get rid of

In [ ]:
# remove the dollar sign before price
combined_df['price'] = combined_df.price.str.replace("\$|,", '').astype(float)

# 6. Imputing data 2

In [ ]:
# Explore missing data
combined_df.isna().sum()

#### Need to fill in bedrooms , bathrooms 1, and reviews to 0. Rationale: Who would hire an airbnb without atleast a bed, or a bathroom. Reviews are NaNs anyway

In [ ]:
combined_df['bathrooms'].fillna(value=1, inplace=True)  # 1 bathroom
combined_df['reviews_per_month'].fillna(value=0, inplace=True)  # reviews put to 0

In [ ]:
combined_df.isna().sum()

#### Now only bedrooms to do. I am going to order my accomodate and use bfill so ones with similar accomodation levels will have similar levels of bedrooms logically

In [ ]:
# sort by accomodates
combined_df = combined_df.sort_values(by='accommodates', ascending=False)  

# now fill the empty bedrooms using bfill
combined_df['bedrooms'].fillna(axis=0, method='bfill', inplace=True)

In [ ]:
combined_df.isna().sum()

#### No more empty values!!!

# 7.Manually encode amenities

#### The data has a list of amenities. However, they're in list in a column so will need to manually One hot Encode them.

In [ ]:
combined_df.amenities.tolist()

In [ ]:
amenities = ["wifi", "kitchen", "parking", "tv", "wash", "washer", "garden", "balcony"]

# make zero columns for selected amenities, to be one hot encoded manually
for amenity in amenities:
    combined_df["has_" + amenity] = 0

In [ ]:
# Use loc and Regex to manually OHE
# Check list of amenities to see if desired is mentioned

for amenity in amenities:
    contains_mask = combined_df['amenities'].str.lower().str.contains(amenity)
    combined_df.loc[contains_mask, "has_" + amenity] = 1

In [ ]:
combined_df

# 8. Combining Supplementary data

### We will use a lookup table that links longitude and latitudes to [lower layer super output areas (LSOA)'s](https://www.ons.gov.uk/methodology/geography/ukgeographies/censusgeography). This will allow us to join the location of each property to other data.

This look up table was created using a spatial join, seeing which points (properties) exist within the polygons (LSOAs). However, as geospatial work is not part of this course, the resulting table has been given instead.

## 8.1 Loading LSOA lat/long lookup table

In [ ]:
# Access the data LSOA lookup table
lsoa = pd.read_csv("../../data/airbnb/lsoa/lat_lon_lsoa_join.csv", usecols=[1,2,3])
lsoa

## 8.2. Merging index of multiple deprvation (IMD) data and median house price data to shape data (including minor wrangling). Can be found at lsoa level online.

In [ ]:
# read imd and median house price data
imd = pd.read_csv("..//..//data//airbnb//imd//imd.csv")
median_house_price = pd.read_csv("..//..//data//airbnb//house_prices//median_house_lsoa_2020.csv")

In [ ]:
#rename columns in place
imd.rename(columns=({"LSOA code (2011)":"LSOA11CD"}), inplace=True)
median_house_price.rename(columns=({"LSOA code":"LSOA11CD",
                                    "Year ending Jun 2020": "median_house_price_2020"}), 
                          inplace=True)

In [ ]:
#no idea why median house price has 2 empty columns, but I'm going to drop them
median_house_price.drop(columns=["Unnamed: 5",'Unnamed: 6'], inplace=True)

In [ ]:
# annoyingly empty values have a :, so one extra step before dropping empty values
median_house_price['median_house_price_2020'] = (median_house_price['median_house_price_2020']
                                                 .str.replace(':', 'NaN'))
median_house_price = median_house_price.dropna()  # dropnas

In [ ]:
#remove the comma from price
median_house_price['median_house_price_2020'] = (median_house_price['median_house_price_2020']
                                                 .str.replace(',', '')
                                                 .astype(float))
# Remove remaining missing values
median_house_price = median_house_price.dropna()

In [ ]:
# merge imd and house prices
imd_median_house = imd.merge(median_house_price, on="LSOA11CD")

In [ ]:
imd_median_house = imd_median_house[['LSOA11CD', 'Index of Multiple Deprivation (IMD) Score',
                                     'median_house_price_2020']]  # subset required columns

In [ ]:
lsoa

In [ ]:
imd_median_geography = lsoa.merge(imd_median_house, on='LSOA11CD')

In [ ]:
imd_median_geography

## 8.3.Journey time Data (From department from transport)

In [ ]:
# reading the data
journey_town = pd.read_csv(
    "..//..//data//airbnb//journey_data//min_town_centre_ptw.csv")
journey_station = pd.read_csv(
    "..//..//data//airbnb//journey_data//avg_time_rail_station_ptw.csv")
journey_airport = pd.read_csv(
    "..//..//data//airbnb//journey_data//avg_time_airport_ptw.csv")

In [ ]:
# subset columns and change column name for ease of use

# town
journey_town = journey_town.rename(columns=(
    {'mins_nearest_town_pubt_walk': 'minutes_to_town', 'LSOA_code': 'LSOA Code'}))
journey_town = journey_town[['LSOA Code', 'minutes_to_town']]
# station
journey_station = journey_station.rename(
    columns=({'Average travel time to rail station(minutes)': 'minutes_to_rail'}))
journey_station = journey_station[['LSOA Code', 'minutes_to_rail']]
# airport
journey_airport = journey_airport.rename(
    columns=({'Average minimum journey time (minutes)': 'minutes_to_airport'}))
journey_airport = journey_airport[['LSOA Code', 'minutes_to_airport']]

In [ ]:
# Join all the journey data together into one frame
journey_dfs = [journey_town, journey_station, journey_airport]

df_all_journeys = reduce(lambda left, right: pd.merge(
    left, right, on='LSOA Code'), journey_dfs)  # reduce them 2 one dataframe (basically a merge)
df_all_journeys = df_all_journeys.rename(columns=({'LSOA Code': 'LSOA11CD'}))

In [ ]:
df_all_journeys

## 8.4. Merging all supplementary data together

In [ ]:
# Combine transport times, imd and median house price
supp_data=imd_median_geography.merge(df_all_journeys,on='LSOA11CD')

In [ ]:
supp_data

# 9. Join LSOA supplementary data with listings using lat/long

In [ ]:
airbnb_with_supp = combined_df.merge(supp_data, on=["latitude", "longitude"])

In [ ]:
airbnb_with_supp.columns

# 10. Final wranglings 

In [ ]:
#check for nas
airbnb_with_supp.isna().sum()

In [ ]:
airbnb_with_supp=airbnb_with_supp.drop(columns=['amenities','reviews_per_month']) #dont need these

In [ ]:
airbnb_with_supp.dtypes

##### Turn host is superhost to binary and clean out the lettters in the bathrooms (I dont know why they are there, but we just give them a 1)

In [ ]:
airbnb_with_supp['bathrooms'].replace({'H': 1}, inplace=True)
airbnb_with_supp['bathrooms'].replace({'S': 1}, inplace=True)
airbnb_with_supp['bathrooms'].replace({'P': 1}, inplace=True)

In [ ]:
airbnb_with_supp['bathrooms'] = airbnb_with_supp['bathrooms'].astype(float)

In [ ]:
# OHE the room type. Also convert t/f, to binary
airbnb_with_supp = pd.get_dummies(airbnb_with_supp, columns=['room_type'])
airbnb_with_supp = airbnb_with_supp.replace({'t': 1, 'f': 0})

In [ ]:
airbnb_with_supp.columns

In [ ]:
airbnb_with_supp #everything looks fine

In [ ]:
airbnb_with_supp.to_csv('../../data/airbnb/example_cleaned_data.csv', index=False)